You need to run litellm proxy first.

Install ag2 with openai

In [1]:
!pip install ag2[openai]

zsh:1: no matches found: ag2[openai]


or

In [2]:
!pip install 'ag2[openai]'

  Using cached ag2-0.10.0-py3-none-any.whl.metadata (36 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached pydantic-2.12.3-py3-none-any.whl.metadata (87 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached termcolor-3.2.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached tiktoken-0.12.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.11.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.41.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.3 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 773.5 kB/s eta 0:00:00a 0:00:01
Using

For more information about ag2 framework do to [docs](https://docs.ag2.ai/latest/)

In [ ]:
import os
import litellm
from dotenv import load_dotenv
from typing import Annotated, Any
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat.group.patterns import AutoPattern
from autogen.agentchat import initiate_group_chat

Set env variables:
- your litellm proxy master key
- your litellm proxy url
- set use_litellm_proxy True

In [ ]:
litellm.use_litellm_proxy = True
load_dotenv()
api_base = os.getenv("LITELLM_ROXY_URL")
api_key = os.getenv("LITELLM_MASTER_KEY")

Define the tool function

In [ ]:
def get_weather(city: str) -> str:
    """Moke function"""
    city_normalized = city.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"The weather in {city} is sunny with a temperature of 20°C."

Set up the model with your proxy params

In [ ]:
llm_config = LLMConfig(config_list={"model": "sap/gpt-4o", "base_url": api_base, "api_key": api_key})

Define the function for check if agent conversation is finished

In [ ]:
def is_termination_msg(msg: dict[str, Any]) -> bool:
    content = msg.get("content", "")
    return (content is not None) and "==== REPORT GENERATED ====" in content

Set up an agent with created model and tool, put the system message if needed

In [ ]:
assistant = ConversableAgent(name="assistant",
                             llm_config=llm_config,
                             system_message="You are a helpful weather assistant. "
                                            "When the user asks for the weather in a specific city, "
                                            "use the 'get_weather' tool to find the information. "
                                            "If the tool returns an error, inform the user politely. "
                                            "If the tool is successful, write a couple sentences for "
                                            "TV weather report in the city, that will be include small jok."
                                            "Once you've generated the report append the below in the summary:"
                                            "==== REPORT GENERATED ====",
                             functions=[get_weather])

Set up the conversation pattern with created agent, model and function for conversation termination

In [ ]:
pattern = AutoPattern(initial_agent=assistant,
                      agents=[assistant],
                      group_manager_args={
                          "llm_config": llm_config,
                          "is_termination_msg": is_termination_msg
                      },
                      )

Run the conversation with message

In [ ]:
result, _, _ = initiate_group_chat(pattern=pattern,
                                   messages="What is the weather like in Tbilisi?",
                                   )